In [ ]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", 'main')
print(gsm8k)
print(gsm8k['train'][0]['question'])
print(gsm8k['train'][0]['answer'])

In [ ]:
# Import several open-source LLMs
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
# model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf')
tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")
model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.5").to('cuda:1')

In [ ]:
# Try answering the first question
input_ids = tokenizer.encode(gsm8k['train'][0]['question'], return_tensors='pt')
sample_output = model.generate(input_ids, do_sample=True, top_k=50, max_length=1000, top_p=0.95, num_return_sequences=1)
print("Response:", tokenizer.decode(sample_output[0], skip_special_tokens=True))
print("Answer:", gsm8k['train'][0]['answer'])